In [60]:
import spacy
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from imblearn.under_sampling import RandomUnderSampler 

## Building Bag of Words

In [27]:
nlp = spacy.load('pt_core_news_sm')
stopwords = nlp.Defaults.stop_words

In [28]:
vectorizer = CountVectorizer(lowercase=True, stop_words=stopwords)

In [43]:
df = pd.read_csv('data/reviews_final.csv')
df.head()

,title,review,rating,date,label
0,Ótimo custo benefício.,Até agora estou amando o aparelho. Estou bem s...,5.0,25/10/2019,Good
1,Não recebi a versão global e produto veio sem ...,"Adquiri o produto em 09/11, chegou hoje e para...",1.0,14/11/2019,Bad
2,"Bom, bonito e barato. Porém sem nota fiscal!",Comprei o celular no sábado dia 26/10/19 e che...,5.0,31/10/2019,Good
3,"Bom, no prazo, carregador fora do padrão brasi...","Chegou dentro do prazo, bem embalado. Tudo ok,...",3.0,19/11/2019,Neutral
4,Celular acima das expectativas,O celular superou totalmente minhas expectativ...,5.0,29/10/2019,Good


In [44]:
df = df.dropna()
df.isnull().sum()

title     0
review    0
rating    0
date      0
label     0
dtype: int64

In [47]:
df = df.query('label != "Neutral"')
df.label.unique()

array(['Good', 'Bad'], dtype=object)

In [59]:
df.label.value_counts()

Good    4448
Bad      192
Name: label, dtype: int64

### Undersampling

In [63]:
sampler = RandomUnderSampler()

X = df[['title', 'review']]
y = df['label']

X_res, y_res = sampler.fit_resample(X, y)
y_res.value_counts()

Bad     192
Good    192
Name: label, dtype: int64

In [64]:
X = X_res['review']
y = df['label']

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score


vectorizer = TfidfVectorizer(stop_words=stopwords)
X_transformed = vectorizer.fit_transform(X)

true_k = 2
model = KMeans(n_clusters=true_k, max_iter=100, random_state=42)
model.fit(X_transformed)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(true_k):
    print("\nCluster %d:" % i),
    
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),

Top terms per cluster:

Cluster 0:
 celular
 produto
 aparelho
 bateria
 chegou
 excelente
 câmera
 ótimo
 qualidade
 entrega

Cluster 1:
 nota
 fiscal
 produto
 veio
 carregador
 padrão
 garantia
 brasileiro
 vendedor
 chegou
